# Step 4: Assessment (Granular) with Bounding Boxes

This notebook demonstrates the **granular assessment** approach with **automatic bounding box processing** for evaluating extraction confidence using AWS Bedrock.

**Key Features:**
- Multiple focused inferences instead of single large inference
- **Automatic spatial localization with bounding boxes**
- **Visual annotation of extracted fields**
- Prompt caching for cost optimization
- Parallel processing for reduced latency
- Better handling of complex documents with many attributes

**Inputs:**
- Document object with extraction results from Step 3
- Granular assessment configuration with enhanced prompts
- Document classes with confidence thresholds

**Outputs:**
- Document with enhanced assessment results including geometry data
- Detailed confidence scores and reasoning for each attribute
- **Bounding box coordinates for spatial localization**
- **Visual annotation of document pages with extracted fields**
- Performance metrics showing granular processing benefits

## 0. Package Installation

First, let's ensure we have the latest version of the IDP common package with bounding box support:

In [ ]:
ROOTDIR="../.."

# Let's make sure that modules are autoreloaded
%load_ext autoreload
%autoreload 2

# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[dev, all]"

# Check installed version
%pip show idp_common | grep -E "Version|Location"

## 1. Load Previous Step Data

In [ ]:
import os
import json
import time
import logging
import boto3
import yaml
from pathlib import Path

# Import IDP libraries
from idp_common.models import Document, Status
from idp_common import assessment

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import io

# Configure logging
logging.basicConfig(level=logging.WARNING)
logging.getLogger('idp_common.assessment.granular_service').setLevel(logging.INFO)
logging.getLogger('idp_common.bedrock.client').setLevel(logging.INFO)

print("Libraries imported successfully")
print("Granular assessment with automatic bounding box support enabled")

In [ ]:
# Load document from previous step
extraction_data_dir = Path(".data/step3_extraction")

# Load document object from JSON
document_path = extraction_data_dir / "document.json"
with open(document_path, 'r') as f:
    document = Document.from_json(f.read())

# Load configuration - use enhanced config with bounding boxes
config_dir = Path("config")
CONFIG = {}

config_files = [
    "assessment_with_bounding_boxes.yaml",  # Enhanced config with spatial prompts
    "classes.yaml"
]

# Override to enable granular assessment
for config_file in config_files:
    config_path = config_dir / config_file
    if config_path.exists():
        with open(config_path, 'r') as f:
            file_config = yaml.safe_load(f)
            CONFIG.update(file_config)
        print(f"Loaded {config_file}")
    else:
        print(f"Warning: {config_file} not found")

# Enable granular assessment
if 'assessment' not in CONFIG:
    CONFIG['assessment'] = {}
if 'granular' not in CONFIG['assessment']:
    CONFIG['assessment']['granular'] = {}
CONFIG['assessment']['granular']['enabled'] = True

# Load environment info
env_path = extraction_data_dir / "environment.json"
with open(env_path, 'r') as f:
    env_info = json.load(f)

# Set environment variables
os.environ['AWS_REGION'] = env_info['region']
os.environ['METRIC_NAMESPACE'] = 'IDP-Modular-Pipeline'

print(f"Loaded document: {document.id}")
print(f"Document status: {document.status.value}")
print(f"Number of sections: {len(document.sections) if document.sections else 0}")
print(f"Configuration sections: {list(CONFIG.keys())}")

## 2. Configure Granular Assessment Service

In [ ]:
# Extract assessment configuration
assessment_config = CONFIG.get('assessment', {})
granular_config = assessment_config.get('granular', {})

print("=== Assessment Configuration ===")
print(f"Model: {assessment_config.get('model')}")
print(f"Temperature: {assessment_config.get('temperature')}")
print(f"Default Confidence Threshold: {assessment_config.get('default_confidence_threshold')}")

print("\n=== Granular Configuration ===")
print(f"Enabled: {granular_config.get('enabled', False)}")
print(f"Max Workers: {granular_config.get('max_workers', 4)}")
print(f"Simple Batch Size: {granular_config.get('simple_batch_size', 3)}")
print(f"List Batch Size: {granular_config.get('list_batch_size', 1)}")

print("\n=== Enhanced Bounding Box Prompts ===")
task_prompt = assessment_config.get('task_prompt', '')
has_spatial_guidelines = 'spatial-localization-guidelines' in task_prompt
has_bbox_instructions = 'bbox:' in task_prompt
print(f"Has spatial localization guidelines: {has_spatial_guidelines}")
print(f"Has bounding box instructions: {has_bbox_instructions}")

In [ ]:
# Create assessment service - will automatically use granular if enabled
assessment_service = assessment.AssessmentService(config=CONFIG)

print(f"Assessment service initialized: {type(assessment_service._service).__name__}")
service_type = 'Granular' if 'Granular' in type(assessment_service._service).__name__ else 'Original'
print(f"Service type: {service_type}")
print("✅ Automatic bounding box processing enabled for granular assessment")

## 3. Helper Functions

In [ ]:
def parse_s3_uri(uri):
    parts = uri.replace("s3://", "").split("/")
    bucket = parts[0]
    key = "/".join(parts[1:])
    return bucket, key

def load_json_from_s3(uri):
    s3_client = boto3.client('s3')
    bucket, key = parse_s3_uri(uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    content = response['Body'].read().decode('utf-8')
    return json.loads(content)

def load_image_from_s3(uri):
    s3_client = boto3.client('s3')
    bucket, key = parse_s3_uri(uri)
    response = s3_client.get_object(Bucket=bucket, Key=key)
    image_data = response['Body'].read()
    return Image.open(io.BytesIO(image_data))

print("Helper functions defined")

## 4. Assess Extraction Results with Granular Approach

In [ ]:
print("Assessing extraction confidence using granular approach with bounding boxes...")

if not document.sections:
    print("No sections found in document. Cannot proceed with assessment.")
else:
    assessment_results = []
    
    # Process each section that has extraction results (limit to first 2)
    sections_with_extractions = [s for s in document.sections if hasattr(s, 'extraction_result_uri') and s.extraction_result_uri]
    n = min(2, len(sections_with_extractions))
    
    print(f"Found {len(sections_with_extractions)} sections with extraction results")
    print(f"Processing first {n} sections for granular assessment...")
    
    for i, section in enumerate(sections_with_extractions[:n]):
        print(f"\n--- Granular Assessment: Section {i+1}/{n} ---")
        print(f"Section ID: {section.section_id}")
        print(f"Classification: {section.classification}")
        
        # Process section assessment
        start_time = time.time()
        document = assessment_service.process_document_section(
            document=document,
            section_id=section.section_id
        )
        assessment_time = time.time() - start_time
        
        print(f"✅ Granular assessment completed in {assessment_time:.2f} seconds")
        
        # Show granular performance metrics
        try:
            updated_extraction_data = load_json_from_s3(section.extraction_result_uri)
            metadata = updated_extraction_data.get('metadata', {})
            
            if metadata.get('granular_assessment_used'):
                print(f"📊 Granular Tasks: {metadata.get('assessment_tasks_total', 'N/A')} total")
                print(f"   ✅ Successful: {metadata.get('assessment_tasks_successful', 'N/A')}")
                print(f"   ❌ Failed: {metadata.get('assessment_tasks_failed', 'N/A')}")
        except Exception as e:
            print(f"Could not load granular metadata: {e}")
        
        assessment_results.append({
            'section_id': section.section_id,
            'classification': section.classification,
            'processing_time': assessment_time,
            'extraction_result_uri': section.extraction_result_uri
        })
    
    print(f"\n🎉 Granular assessment with bounding boxes complete for {n} sections!")

## 5. Display Assessment Results with Spatial Data

In [ ]:
def display_granular_assessment_with_geometry(data, attr_name="", indent="  "):
    if isinstance(data, dict):
        if 'confidence' in data:
            confidence = data.get('confidence', 0)
            threshold = data.get('confidence_threshold', 0.9)
            reason = data.get('confidence_reason', 'No reason')
            geometry = data.get('geometry', [])
            
            status = "✅" if confidence >= threshold else "⚠️"
            print(f"{indent}{status} {attr_name}: {confidence:.3f} (threshold: {threshold})")
            print(f"{indent}   {reason[:100]}{'...' if len(reason) > 100 else ''}")
            
            if geometry:
                bbox = geometry[0]['boundingBox']
                page = geometry[0]['page']
                print(f"{indent}   📍 Page {page}: {bbox['top']*100:.1f}%,{bbox['left']*100:.1f}% ({bbox['width']*100:.1f}%×{bbox['height']*100:.1f}%)")
        else:
            print(f"{indent}{attr_name} (Group):")
            for k, v in data.items():
                display_granular_assessment_with_geometry(v, k, indent + "  ")
    elif isinstance(data, list):
        print(f"{indent}{attr_name} (List - {len(data)} items):")
        for i, item in enumerate(data[:3]):  # Show first 3
            print(f"{indent}  📄 Item {i+1}:")
            for k, v in item.items():
                display_granular_assessment_with_geometry(v, k, indent + "    ")
        if len(data) > 3:
            print(f"{indent}  ... {len(data)-3} more items")

print("Enhanced granular assessment display function defined")

In [ ]:
print("\n=== Granular Assessment Results with Bounding Boxes ===")

geometry_data_by_page = {}

if document.sections:
    sections_with_extractions = [s for s in document.sections if hasattr(s, 'extraction_result_uri') and s.extraction_result_uri]
    
    for section in sections_with_extractions[:2]:
        print(f"\n--- {section.section_id} ({section.classification}) ---")
        
        try:
            extraction_data = load_json_from_s3(section.extraction_result_uri)
            
            # Show granular metrics
            metadata = extraction_data.get('metadata', {})
            if metadata.get('granular_assessment_used'):
                print(f"📊 Granular: {metadata.get('assessment_tasks_total', 0)} tasks in {metadata.get('assessment_time_seconds', 0):.1f}s")
            
            explainability_info = extraction_data.get('explainability_info', [])
            if explainability_info:
                assessments = explainability_info[0]
                
                # Display and collect geometry data
                geometry_count = 0
                for attr_name, attr_data in assessments.items():
                    display_granular_assessment_with_geometry(attr_data, attr_name)
                    
                    # Collect geometry data for visualization
                    if isinstance(attr_data, dict) and 'geometry' in attr_data:
                        geometry_count += 1
                        for geom in attr_data['geometry']:
                            page = geom.get('page', 1)
                            if page not in geometry_data_by_page:
                                geometry_data_by_page[page] = []
                            geometry_data_by_page[page].append({
                                'attr_name': attr_name,
                                'confidence': attr_data.get('confidence', 0),
                                'geometry': geom,
                                'section': section.classification
                            })
                
                print(f"📍 Attributes with spatial data: {geometry_count}/{len(assessments)}")
        except Exception as e:
            print(f"Error: {e}")

print(f"\n📍 Total pages with geometry data: {len(geometry_data_by_page)}")

## 6. Visualize Bounding Boxes from Granular Assessment

In [ ]:
if geometry_data_by_page and document.pages:
    print("\n🎨 Creating granular assessment bounding box visualizations...")
    
    for page_num, bbox_list in geometry_data_by_page.items():
        print(f"\n--- Visualizing Page {page_num} ({len(bbox_list)} fields) ---")
        
        page_id = str(page_num)
        if page_id in document.pages:
            page = document.pages[page_id]
            
            try:
                image = load_image_from_s3(page.image_uri)
                
                # Create visualization
                fig, ax = plt.subplots(1, 1, figsize=(12, 16))
                ax.imshow(image)
                ax.set_title(f"Granular Assessment - Page {page_num}", fontsize=14, fontweight='bold')
                ax.axis('off')
                
                img_width, img_height = image.size
                colors_used = set()
                
                for item in bbox_list:
                    bbox = item['geometry']['boundingBox']
                    confidence = item['confidence']
                    
                    # Convert normalized to pixel coordinates
                    left = bbox['left'] * img_width
                    top = bbox['top'] * img_height
                    width = bbox['width'] * img_width
                    height = bbox['height'] * img_height
                    
                    # Color based on confidence
                    if confidence >= 0.9:
                        color = 'green'
                    elif confidence >= 0.7:
                        color = 'orange'
                    else:
                        color = 'red'
                    colors_used.add(color)
                    
                    # Draw rectangle
                    rect = patches.Rectangle(
                        (left, top), width, height,
                        linewidth=2, edgecolor=color, facecolor='none', alpha=0.8
                    )
                    ax.add_patch(rect)
                    
                    # Add label
                    label = f"{item['attr_name']} ({confidence:.2f})"
                    ax.text(
                        left, max(0, top - 10), label,
                        fontsize=8, color=color, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.2", facecolor='white', alpha=0.8)
                    )
                
                # Add legend
                if colors_used:
                    legend_elements = []
                    if 'green' in colors_used:
                        legend_elements.append(patches.Patch(color='green', label='High Confidence (≥0.9)'))
                    if 'orange' in colors_used:
                        legend_elements.append(patches.Patch(color='orange', label='Medium Confidence (≥0.7)'))
                    if 'red' in colors_used:
                        legend_elements.append(patches.Patch(color='red', label='Low Confidence (<0.7)'))
                    ax.legend(handles=legend_elements, loc='upper right')
                
                plt.tight_layout()
                plt.show()
                
                print(f"✅ Granular assessment visualization: {len(bbox_list)} fields on page {page_num}")
                
            except Exception as e:
                print(f"❌ Error visualizing page {page_num}: {e}")
else:
    print("\n📍 No geometry data available for visualization")
    print("   This could mean the LLM didn't provide bounding box coordinates")

## 7. Granular Performance Analysis

In [ ]:
print("\n=== Granular Performance Analysis with Bounding Boxes ===")

if document.sections:
    sections_with_extractions = [s for s in document.sections if hasattr(s, 'extraction_result_uri') and s.extraction_result_uri]
    
    total_tasks = 0
    total_time = 0
    total_geometry_fields = sum(len(bbox_list) for bbox_list in geometry_data_by_page.values())
    
    for section in sections_with_extractions[:2]:
        try:
            extraction_data = load_json_from_s3(section.extraction_result_uri)
            metadata = extraction_data.get('metadata', {})
            
            if metadata.get('granular_assessment_used'):
                tasks = metadata.get('assessment_tasks_total', 0)
                time_taken = metadata.get('assessment_time_seconds', 0)
                
                total_tasks += tasks
                total_time += time_taken
                
                print(f"\nSection {section.section_id}:")
                print(f"  • Granular tasks: {tasks}")
                print(f"  • Processing time: {time_taken:.2f}s")
                print(f"  • Avg time per task: {time_taken/tasks:.3f}s" if tasks > 0 else "  • No tasks")
        except Exception as e:
            print(f"Error analyzing section {section.section_id}: {e}")
    
    if total_tasks > 0:
        print(f"\n📈 Overall Granular Performance:")
        print(f"  • Total assessment tasks: {total_tasks}")
        print(f"  • Total processing time: {total_time:.2f}s")
        print(f"  • Average time per task: {total_time/total_tasks:.3f}s")
        print(f"  • Fields with spatial data: {total_geometry_fields}")
        print(f"  • Pages with visualizations: {len(geometry_data_by_page)}")
        
        print(f"\n💡 Granular + Bounding Box Benefits:")
        print(f"  • Focused assessments: Each task handles 1-3 attributes")
        print(f"  • Parallel processing: Multiple tasks run concurrently")
        print(f"  • Automatic spatial data: Bbox → geometry conversion in each task")
        print(f"  • Prompt caching: Reduces token costs by 80-90%")
        print(f"  • Visual validation: See exactly where each field is located")
else:
    print("No sections available for performance analysis")

## 8. Save Results for Next Step

In [ ]:
# Create data directory for this step
data_dir = Path(".data/step4_assessment")
data_dir.mkdir(parents=True, exist_ok=True)

# Save updated document object as JSON
document_path = data_dir / "document.json"
with open(document_path, 'w') as f:
    f.write(document.to_json())

# Save configuration (pass through)
config_path = data_dir / "config.json"
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2)

# Save environment info (pass through)
env_path = data_dir / "environment.json"
with open(env_path, 'w') as f:
    json.dump(env_info, f, indent=2)

print(f"Saved document to: {document_path}")
print(f"Saved configuration to: {config_path}")
print(f"Saved environment info to: {env_path}")

## 9. Summary

In [ ]:
sections_assessed = len(assessment_results) if 'assessment_results' in locals() else 0
total_geometry = sum(len(bbox_list) for bbox_list in geometry_data_by_page.values())

print("=== Step 4: Granular Assessment with Bounding Boxes Complete ===")
print(f"✅ Document processed: {document.id}")
print(f"✅ Sections assessed: {sections_assessed}")
print(f"✅ Granular approach used: {granular_config.get('enabled', False)}")
print(f"✅ Fields with spatial data: {total_geometry}")
print(f"✅ Model used: {assessment_config.get('model')}")

print("\n📌 Next step: Run step5_summarization.ipynb")
print("\n📋 Granular Assessment + Bounding Box Features Demonstrated:")
print("  • Multiple focused inferences with spatial localization")
print("  • Automatic bounding box conversion (bbox → geometry)")
print("  • Parallel processing with visual annotation")
print("  • Prompt caching for cost optimization (80-90% savings)")
print("  • Enhanced performance metrics and task breakdown")
print("  • Better handling of complex documents with spatial validation")
print("  • UI-compatible geometry format output")